In [ ]:
import pandas as pd

In [ ]:
# Import Billing File
file_path = (
    r"C:\Users\pirat\Dropbox\Consulting Inc\Upperline Health\1. Payor Data & Contracts"
    r"\99-Athena Reports\ULH All Payors FL - DOS 2024 & YTD June 2025 - as of 11.17.25.xlsx"
)

# Read all sheets (dict: {sheet_name: DataFrame})
sheets_dict = pd.read_excel(file_path, sheet_name=None)

# If the file really only has those 2 relevant sheets, this will grab both
df_raw = pd.concat(sheets_dict.values(), ignore_index=True)

# delete duplicate rows
df_raw = df_raw.drop_duplicates(keep='first')

# BENCHMARK: 1m per 1 mil rows

In [14]:
# Initial cleaning
df_clean = df_raw.copy()

# change net payment from negative to positive
df_clean['Net Payment'] = df_clean['Net Payment'].abs()

# create avg paid per claim
df_clean['Avg_Paid_Per_Claim'] = (df_clean['Net Payment']/df_clean['Charge Units']).fillna(0)

# create key from Claim ID and Patient ID
df_clean["Patient_Claim_Key"] = (
    df_clean["Claim ID"].astype(str)
    + "-"
    + df_clean["Patient ID"].astype(str)
    + "-"
    + df_clean["Date of Service"].astype(str)
)

# Get "State" from first two characters of ["Service Department"]
df_clean["State"] = df_clean["Service Department"].str[:2]

In [ ]:
# Import Payer Crosswalk File
file_path = (
    r"C:\Users\pirat\Dropbox\Consulting Inc\Upperline Health\1. Payor Data & Contracts"
    r"\Payor Mapping_FIN.xlsx"
)

# Read only "Payor Mapping_FIN" sheet
df_payor_mapping = pd.read_excel(file_path, sheet_name="PayorMap_FIN")

# delete duplicate rows
df_payor_mapping = df_payor_mapping.drop_duplicates(keep='first')

In [ ]:
# hopefully a temporary fix for non-matching insurance names in the mapping file

# unique list of "Primary Insurance" values from df_clean
primary_insurance = df_clean["Primary Insurance"].unique()

# Convert to DataFrame for easier handling
primary_insurance = pd.DataFrame(primary_insurance, columns=["Primary Insurance"])

